In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision #数据库模块
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
# Hyper Parameters
EPOCH = 1 #刷新批量
BATCH_SIZE = 50 #
LR = 0.001
DOWNLOAD_MNIST = False

train_data = torchvision.datasets.MNIST(
    root = './mnist'
    ,train = True
    ,transform = torchvision.transforms.ToTensor()#原始的图片数据是numpy的数据
    ,download = DOWNLOAD_MNIST
)# Hyper Parameters
EPOCH = 1 #刷新批量
BATCH_SIZE = 50 #
LR = 0.001
DOWNLOAD_MNIST = False

train_data = torchvision.datasets.MNIST(
    root = './mnist'
    ,train = True
    ,transform = torchvision.transforms.ToTensor()#原始的图片数据是numpy的数据
    ,download = DOWNLOAD_MNIST
)

In [7]:
train_loader = Data.DataLoader(dataset=train_data
                               ,batch_size=BATCH_SIZE
                               ,shuffle=True
                               ,num_workers = 2)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist
    Split: Train
    StandardTransform
Transform: ToTensor()

In [13]:
test_data = torchvision.datasets.MNIST(root='./mnist/'
,train=False)
test_x = Variable(torch.unsqueeze(test_data.test_data,dim=1)
,volatile=True).type(torch.FloatTensor)[:2000]/255
test_y = test_data.test_labels[:2000]
train_data

/home/manman/anaconda3/envs/torch38/lib/python3.8/site-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
<ipython-input-13-a0200599a52b>:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  test_x = Variable(torch.unsqueeze(test_data.test_data,dim=1)
/home/manman/anaconda3/envs/torch38/lib/python3.8/site-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist
    Split: Train
    StandardTransform
Transform: ToTensor()

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d( #输入维度 /维度变化->(1,28,28)
                in_channels = 1 #RGB的高度
                ,out_channels = 16 
                ,kernel_size = 5 # 扫描区域设定，5*5
                ,stride = 1 #扫描的的时候跳跃的步长
                ,padding = 2 #扫描的时候将原始图片增加一层帮助图片滑动窗口时候避免超出范围
                ) #卷积层，筛选特征 /维度变化->(16,28,28)
                #padding参数的计算方法
                #if stride = 1,padding = (kernel_size-1)/2
                ,nn.ReLU() #激活函数 /维度变化->(16,28,28)
                ,nn.MaxPool2d(kernel_size=2) #池化层选择特征中2*2范围内最大的值当作特征 /维度变化->(16,14,14)
                ) 

        self.conv2 = nn.Sequential( #/第二层维度变化->(16,14,14)
            nn.Conv2d(
                in_channels = 16 #RGB的高度接收上次一层的维度16
                ,out_channels = 32
                ,kernel_size = 5 # 扫描区域设定，5*5
                ,stride = 1 #扫描的的时候跳跃的步长
                ,padding = 2 #扫描的时候将原始图片增加一层帮助图片滑动窗口时候避免超出范围
                ) #卷积层，筛选特征 /第二层维度变化->(32,14,14)
                ## padding参数的计算方法
                ## if stride = 1,padding = (kernel_size-1)/2
                ,nn.ReLU() #激活函数 /第二层维度变化->(32,14,14)
                ,nn.MaxPool2d(kernel_size=2) #池化层选择特征中2*2范围内最大的值当作特征 /第二层维度变化->(32,7,7)
                )
    
        self.conv3 = nn.Sequential( #/第二层维度变化->(16,14,14)
            nn.Conv2d(
                in_channels = 32 #RGB的高度接收上次一层的维度16
                ,out_channels = 74
                ,kernel_size = 5 # 扫描区域设定，5*5
                ,stride = 1 #扫描的的时候跳跃的步长
                ,padding = 2 #扫描的时候将原始图片增加一层帮助图片滑动窗口时候避免超出范围
                ) #卷积层，筛选特征 /第二层维度变化->(32,14,14)
                ## padding参数的计算方法
                ## if stride = 1,padding = (kernel_size-1)/2
                ,nn.ReLU() #激活函数 /第二层维度变化->(32,14,14)
                ,nn.MaxPool2d(kernel_size=2) #池化层选择特征中2*2范围内最大的值当作特征 /第二层维度变化->(32,7,7)
                )
        
        self.out = nn.Linear(32*7*7,10) #将上面计算结果的三维数据展平成二维
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x) #->(batch,32,7,7)
        x = x.view(x.size(0),-1) #->(batch,32,*7*7)
        output = self.out(x)
        return output

cnn = CNN()
optimizer = torch.optim.Adam(cnn.parameters(),lr=LR)
loss_func = nn.CrossEntropyLoss() #调用交叉熵函数

for epoch in range(EPOCH):
    for step,(x,y) in enumerate(train_loader): 
        b_x , b_y = Variable(x) , Variable(y)
        output = cnn(b_x) #载入训练数据
        loss = loss_func(output,b_y) #载入训练标签
        optimizer.zero_grad() #梯度清零
        loss.backward() #反向传播
        optimizer.step() #传入步长
        #打印损失函数变化
        if step % 50 == 0:
            test_output = cnn(test_x)
            pred_y = torch.max(test_output,1)[1].data.squeeze()
            accuracy = (pred_y==test_y).numpy().sum()/test_y.size(0)
            print('Epoch:',epoch,'| train loss %4f' % float(loss.data),'| test accuracy:',accuracy,'%')

Epoch: 0 | train loss 2.311535 | test accuracy: 0.1295 %
Epoch: 0 | train loss 0.366605 | test accuracy: 0.842 %
Epoch: 0 | train loss 0.261416 | test accuracy: 0.8945 %
Epoch: 0 | train loss 0.297192 | test accuracy: 0.897 %
Epoch: 0 | train loss 0.103045 | test accuracy: 0.934 %
Epoch: 0 | train loss 0.048671 | test accuracy: 0.9365 %
Epoch: 0 | train loss 0.106471 | test accuracy: 0.9475 %
Epoch: 0 | train loss 0.068121 | test accuracy: 0.952 %
Epoch: 0 | train loss 0.136353 | test accuracy: 0.955 %
Epoch: 0 | train loss 0.109795 | test accuracy: 0.964 %
Epoch: 0 | train loss 0.185315 | test accuracy: 0.962 %
Epoch: 0 | train loss 0.202099 | test accuracy: 0.968 %
Epoch: 0 | train loss 0.083856 | test accuracy: 0.9575 %
Epoch: 0 | train loss 0.057519 | test accuracy: 0.9705 %
Epoch: 0 | train loss 0.160105 | test accuracy: 0.9755 %
Epoch: 0 | train loss 0.080944 | test accuracy: 0.971 %
Epoch: 0 | train loss 0.034534 | test accuracy: 0.9755 %
Epoch: 0 | train loss 0.064747 | test ac

In [6]:
CNN()

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 74, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)